In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [62]:
df = {}

for i in range(2010, 2022):
    df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
    df[i].drop(labels=df[i][df[i]['아내지역코드'] > 39].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편지역코드'] > 39].index, inplace=True)

C:\Users\science7\AppData\Local\Temp\ipykernel_4792\1271597377.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\science7\AppData\Local\Temp\ipykernel_4792\1271597377.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\science7\AppData\Local\Temp\ipykernel_4792\1271597377.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\science7\AppData\Local\Temp\ipykernel_4792\1271597377.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")


In [63]:
for year in range(2010, 2022):
    df[year].loc[df[year]['남편연령'] > df[year]['아내연령'], "결혼 연령차 유형"] = "남성 연상혼" 
    df[year].loc[df[year]['남편연령'] < df[year]['아내연령'], "결혼 연령차 유형"] = "여성 연상혼" 
    df[year].loc[df[year]['남편연령'].astype(int) == df[year]['아내연령'].astype(int), "결혼 연령차 유형"] = "동갑혼" 

    df[year].loc[df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), "아내 기준 대도시 유무"] = "대도시 거주" 
    df[year].loc[~df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), "아내 기준 대도시 유무"] = "대도시 비거주" 

    df[year].loc[df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]), "남편 기준 대도시 유무"] = "대도시 거주" 
    df[year].loc[~df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]), "남편 기준 대도시 유무"] = "대도시 비거주" 

    #여성 대도시 거주 유무와 결혼 연령차 유형 연관성 높음

In [73]:
crosstab_years = {}




for year in range(2010, 2022):
    crosstab_years[year] = pd.crosstab(df[year]['남편 기준 대도시 유무'], df[year]['결혼 연령차 유형'], margins=True)

In [82]:


c, p, dof, expected = scipy.stats.chi2_contingency(crosstab_years[2018])

In [83]:
print("c-val : " + str(c))
print("P-val : " + str(p))
print("c\dof : " + str(dof) )

c-val : 40.7511968213106
P-val : 3.241500286033096e-07
c\dof : 6
